In [47]:
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS"
import urllib.request
import logging
import pandas as pd
from selenium.webdriver.support.ui import Select
from re import search


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import requests
import shutil
import time
import os
from pyshadow.main import Shadow
from py7zr import unpack_7zarchive
from RequestInferSchemaToJsonAPI.main import TriggerInferShemaToJsonAPI

In [134]:
logging.basicConfig(handlers=[logging.FileHandler("\\\\10.30.31.77\\data_collection_dump\\TACFolder\\ScraperLogs\\UNAIDS_out.log"),
                            logging.StreamHandler()], level=logging.INFO)
chrome_options = webdriver.ChromeOptions()
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS" # local, gets current working directory
prefs = {'download.default_directory' : base_path}#, "profile.content_settings.exceptions.automatic_downloads.*.setting" : 1}
chrome_options.add_experimental_option('prefs', prefs)
#chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.maximize_window()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome 113.0.5672
INFO:WDM:Driver [C:\Users\10235555\.wdm\drivers\chromedriver\win32\113.0.5672\chromedriver.exe] found in cache


In [122]:
driver.get('https://hivfinancial.unaids.org/hivfinancialdashboards.html')

In [74]:
two_chart = ['Share of domestic resources in Health and AIDS funding','Prevention and treatment interventions for HIV','Trends in international disbursments by source','Trends in international resources for HIV per capita','Health expenditure as a % of GDP','% of general government expenditure to Health','% of general Health expenditure to HIV']
two_chat_nosuccessive = ['Spending on prevention and treatment interventions','Spending on the focused prevention pillars','Spending on key populations (disaggregated by each key population)','Spending on human rights programmes','Spending on PrEP']

In [112]:
from selenium.webdriver.common.action_chains import ActionChains
two_chart = ['Share of domestic resources in Health and AIDS funding','Prevention and treatment interventions for HIV','Trends in international disbursments by source','Trends in international resources for HIV per capita','Health expenditure as a % of GDP','% of general government expenditure to Health','% of general Health expenditure to HIV']
two_chat_nosuccessive = ['Spending on prevention and treatment interventions','Spending on the focused prevention pillars','Spending on key populations (disaggregated by each key population)','Spending on human rights programmes','Spending on PrEP']
country = driver.find_element(By.ID,'countryAHref')
country.click()
time.sleep(1)
select_element = driver.find_element(By.ID,'lstCountries')
select = Select(select_element)
option_list = select.options
for i in option_list[1:]:
    select.select_by_visible_text(i.text)
    time.sleep(2)
    content = driver.find_element(By.CLASS_NAME,'ui.five.column.relaxed.equal.height.divided.stackable.grid')
    list_links = content.find_elements(By.CLASS_NAME,'ui.link.list')
    for l in list_links:
        ind = list_links.index(l)
        topic = content.find_elements(By.TAG_NAME,'h4')[ind].text
        items = l.find_elements(By.CLASS_NAME,'item')
        for item in items:
            dataset = item.text
            if dataset in two_chart:
                item.click()
                time.sleep(5)
                iframe = driver.find_element(By.ID,'frameIt')
                driver.switch_to.frame(iframe)
                # try:
                message_err = driver.find_elements(By.CLASS_NAME,'error-message')[1] #(1,2)
                if message_err == 'Query returned no matching rows.':
                    pass
                else:
                    try:
                        m= driver.find_elements(By.CLASS_NAME,'title.variable-left-margin.with-description')[0]
                    except:
                        m = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin')[1]
                    js_code = "arguments[0].scrollIntoView();"
                    driver.execute_script(js_code, m)
                    # except:
                    #         try:
                    #             m= driver.find_elements(By.CLASS_NAME,"title.variable-left-margin")[1]
                    #             js_code = "arguments[0].scrollIntoView();"
                    #             driver.execute_script(js_code, m)
                    #         except:
                    #             m= driver.find_element(By.CLASS_NAME,"title.variable-left-margin")
                    #             js_code = "arguments[0].scrollIntoView();"
                    #             driver.execute_script(js_code, m)
                    title = m.text
                    m.click()
                    try:
                        more_options = driver.find_elements(By.NAME,'expand')[1]
                        more_options.click()
                    except:
                        more_options = driver.find_elements(By.NAME,'expand')[0]
                        more_options.click()
                    try:
                        source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data source')]")[0].text
                    except:
                        source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data Source')]")[0].text
                    time.sleep(1)
                    download = driver.find_element(By.XPATH,"//*[contains(text(), 'Download Data')]")#more_options.find_elements(By.CLASS_NAME,'text')[1]
                    download.click()
                time.sleep(1)
                message_err_2 = driver.find_elements(By.CLASS_NAME,'error-message')[2].text #(1,2)
                if message_err_2 == 'Query returned no matching rows.':
                    pass
                else:
                    try:
                        m_2 = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin.with-description')[1]
                    except:
                        m_2 = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin')[2]
                    js_code = "arguments[0].scrollIntoView();"
                    driver.execute_script(js_code, m_2)
                    time.sleep(1)
                    title = m_2.text
                    m_2.click()
                    try:
                        more_options_2 = driver.find_elements(By.NAME,'expand')[2]
                        more_options_2.click()
                    except:
                        more_options_2 = driver.find_elements(By.NAME,'expand')[1]
                        more_options_2.click()
                    time.sleep(1)
                    download_2 = driver.find_element(By.XPATH,"//*[contains(text(), 'Download Data')]")#more_options.find_elements(By.CLASS_NAME,'text')[1]
                    download_2.click()
                    try:
                        source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data source')]")[1].text
                    except:
                        source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data Source')]")[1].text
                time.sleep(1)
                driver.switch_to.default_content()
                country = driver.find_element(By.ID,'countryAHref')
                country.click()
                time.sleep(2)
            elif dataset in two_chat_nosuccessive:
                item.click()
                time.sleep(5)
                iframe = driver.find_element(By.ID,'frameIt')
                driver.switch_to.frame(iframe)
                message_err = driver.find_elements(By.CLASS_NAME,'error-message')[1].text #(1,2)
                if message_err == 'Query returned no matching rows.':
                    pass
                else:
                # try:
                    try:
                        m= driver.find_elements(By.CLASS_NAME,'title.variable-left-margin.with-description')[0]
                    except:
                        m = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin')[1]
                    js_code = "arguments[0].scrollIntoView();"
                    driver.execute_script(js_code, m)
                    # except:
                    #         try:
                    #             m= driver.find_elements(By.CLASS_NAME,"title.variable-left-margin")[1]
                    #             js_code = "arguments[0].scrollIntoView();"
                    #             driver.execute_script(js_code, m)
                    #         except:
                    #             m= driver.find_element(By.CLASS_NAME,"title.variable-left-margin")
                    #             js_code = "arguments[0].scrollIntoView();"
                    #             driver.execute_script(js_code, m)
                    title = m.text
                    m.click()
                    try:
                        more_options = driver.find_elements(By.NAME,'expand')[1]
                        more_options.click()
                    except:
                        more_options = driver.find_elements(By.NAME,'expand')[0]
                        more_options.click()
                    time.sleep(1)
                    download = driver.find_element(By.XPATH,"//*[contains(text(), 'Download Data')]")#more_options.find_elements(By.CLASS_NAME,'text')[1]
                    download.click()
                    try:
                        source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data source')]")[0].text
                    except:
                        source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data Source')]")[0].text
                time.sleep(1)
                message_err_2 = driver.find_elements(By.CLASS_NAME,'error-message')[2].text #(1,2)
                if message_err_2 == 'Query returned no matching rows.':
                    pass
                else:
                    try:
                        m_2 = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin.with-description')[3]
                    except:
                        m_2 = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin')[3]
                    js_code = "arguments[0].scrollIntoView();"
                    driver.execute_script(js_code, m_2)
                    title = m_2.text
                    m_2.click()
                    try:
                        more_options_2 = driver.find_elements(By.NAME,'expand')[2]
                        more_options_2.click()
                    except:
                        try:
                            more_options_2 = driver.find_elements(By.NAME,'expand')[3]
                            more_options_2.click()
                        except:
                            more_options_2 = driver.find_elements(By.NAME,'expand')[1]
                            more_options_2.click()
                    time.sleep(1)
                    download_2 = driver.find_element(By.XPATH,"//*[contains(text(), 'Download Data')]")#more_options.find_elements(By.CLASS_NAME,'text')[1]
                    download_2.click()
                    try:
                        source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data source')]")[1].text
                    except:
                        source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data Source')]")[1].text
                time.sleep(1)
                driver.switch_to.default_content()
                country = driver.find_element(By.ID,'countryAHref')
                country.click()
                time.sleep(2)
            else: 
                item.click()
                time.sleep(5)
                iframe = driver.find_element(By.ID,'frameIt')
                driver.switch_to.frame(iframe)
                try:
                    message_err = driver.find_elements(By.CLASS_NAME,'error-message')[1].text 
                except:
                    message_err = driver.find_elements(By.CLASS_NAME,'error-message')[0].text
                if message_err == 'Query returned no matching rows.':
                    pass
                else:
                    try:
                        m= driver.find_element(By.CLASS_NAME,'title.variable-left-margin.with-description')
                        js_code = "arguments[0].scrollIntoView();"
                        driver.execute_script(js_code, m)
                    except:
                            try:
                                m= driver.find_elements(By.CLASS_NAME,"title.variable-left-margin")[1]
                                js_code = "arguments[0].scrollIntoView();"
                                driver.execute_script(js_code, m)
                            except:
                                m= driver.find_element(By.CLASS_NAME,"title.variable-left-margin")
                                js_code = "arguments[0].scrollIntoView();"
                                driver.execute_script(js_code, m)
                    title = m.text
                    m.click()
                    try:
                        more_options = driver.find_elements(By.NAME,'expand')[1]
                        more_options.click()
                    except:
                        more_options = driver.find_elements(By.NAME,'expand')[0]
                        more_options.click()
                    time.sleep(1)
                    download = driver.find_element(By.XPATH,"//*[contains(text(), 'Download Data')]")#more_options.find_elements(By.CLASS_NAME,'text')[1]
                    download.click()
                    try:
                        source = driver.find_element(By.XPATH,"//*[contains(text(), 'Data source')]").text
                    except:
                        source = driver.find_element(By.XPATH,"//*[contains(text(), 'Data Source')]").text
                    time.sleep(1)
                driver.switch_to.default_content()
                country = driver.find_element(By.ID,'countryAHref')
                country.click()
                time.sleep(2)
            # content = driver.find_element(By.CLASS_NAME,'ui.five.column.relaxed.equal.height.divided.stackable.grid')
            # topic = content.find_element(By.TAG_NAME,'h4')[list_links.index(l)].text
            items = l.find_elements(By.CLASS_NAME,'item')
        content = driver.find_element(By.CLASS_NAME,'ui.five.column.relaxed.equal.height.divided.stackable.grid')
        list_links = content.find_elements(By.CLASS_NAME,'ui.link.list')
    
    # select_element = driver.find_element(By.ID,'lstCountries')
    # select = Select(select_element)
    # option_list = select.options[1:]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[contains(text(), 'Download Data')]"}
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00FE8893+48451]
	(No symbol) [0x00F7B8A1]
	(No symbol) [0x00E85058]
	(No symbol) [0x00EB0467]
	(No symbol) [0x00EB069B]
	(No symbol) [0x00EDDD92]
	(No symbol) [0x00ECA304]
	(No symbol) [0x00EDC482]
	(No symbol) [0x00ECA0B6]
	(No symbol) [0x00EA7E08]
	(No symbol) [0x00EA8F2D]
	GetHandleVerifier [0x01248E3A+2540266]
	GetHandleVerifier [0x01288959+2801161]
	GetHandleVerifier [0x0128295C+2776588]
	GetHandleVerifier [0x01072280+612144]
	(No symbol) [0x00F84F6C]
	(No symbol) [0x00F811D8]
	(No symbol) [0x00F812BB]
	(No symbol) [0x00F74857]
	BaseThreadInitThunk [0x76720099+25]
	RtlGetAppContainerNamedObjectPath [0x77657B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77657B3E+238]


In [164]:
driver.get('https://hivfinancial.unaids.org/hivfinancialdashboards.html')

In [165]:
from selenium.webdriver.common.action_chains import ActionChains
country = driver.find_element(By.ID,'countryAHref')
country.click()
time.sleep(1)
select_element = driver.find_element(By.ID,'lstCountries')
select = Select(select_element)
option_list = select.options
for i in option_list[1:]:
    c = i.text
    select.select_by_visible_text(i.text)
    time.sleep(2)
    content = driver.find_element(By.CLASS_NAME,'ui.five.column.relaxed.equal.height.divided.stackable.grid')
    list_links = content.find_elements(By.CLASS_NAME,'ui.link.list')
    for l in list_links:
        ind = list_links.index(l)
        topic = content.find_elements(By.TAG_NAME,'h4')[ind].text
        items = l.find_elements(By.CLASS_NAME,'item')
        for item in items:
            dataset = item.text
            item.click()
            time.sleep(5)
            iframe = driver.find_element(By.ID,'frameIt')
            driver.switch_to.frame(iframe)
            boxes = driver.find_elements(By.CLASS_NAME,'widget-container.dashboard-item-container')
            for box in boxes:
                j = boxes.index(box)
                if len(box.find_elements(By.CLASS_NAME, 'title.variable-left-margin.with-description'))>0 or len(box.find_elements(By.CLASS_NAME, 'title.variable-left-margin'))>0:
                    try :
                        m = box.find_element(By.CLASS_NAME, 'title.variable-left-margin.with-description')
                    except :
                        try :
                            m = box.find_element(By.CLASS_NAME, 'title.variable-left-margin')
                        except:
                            pass
                    js_code = "arguments[0].scrollIntoView();"
                    driver.execute_script(js_code, m)
                    if  box.find_element(By.CLASS_NAME,'error-message').text == 'Query returned no matching rows.' :
                        pass
                    else:
                        title = m.text
                        if title == '':
                            pass
                        elif f'{c} {title}' in os.listdir(base_path):
                            pass
                        else:
                            m.click()
                            more_options = box.find_element(By.NAME,'expand')
                            more_options.click()
                            time.sleep(1)
                            download = driver.find_element(By.XPATH,"//*[contains(text(), 'Download Data')]")
                            download.click()
                            time.sleep(2)
                            def latest_download_file():
                                path = base_path
                                os.chdir(path)
                                files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
                                newest = files[-1]

                                return newest

                            fileends = "crdownload"
                            while "crdownload" == fileends:
                                time.sleep(3) 
                                newest_file = latest_download_file()
                                if "crdownload" in newest_file:
                                    fileends = "crdownload"
                                    # time.sleep(5)
                                else:
                                    fileends = "None"
                            latest_download_file() 
                            file = max([base_path + "\\" + f for f in os.listdir(base_path)],key=os.path.getctime)
                            title = title.replace('/',' ').replace(':',' ').replace('"',' ').replace('*',' ')
                            try:
                                os.rename(file,f'{base_path}\\{c} {title}.csv')
                            except  Exception as err:
                                print(err)
                            try:
                                source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data source')]")[j+1].text
                            except:
                                try:
                                    source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data Source')]")[j+1].text
                                except:
                                    source = ''
            driver.switch_to.default_content()
            country = driver.find_element(By.ID,'countryAHref')
            country.click()
            time.sleep(2)
            items = l.find_elements(By.CLASS_NAME,'item')
        content = driver.find_element(By.CLASS_NAME,'ui.five.column.relaxed.equal.height.divided.stackable.grid')
        list_links = content.find_elements(By.CLASS_NAME,'ui.link.list')
            # message_err = driver.find_elements(By.CLASS_NAME,'error-message')[1] #(1,2)
            # if message_err == 'Query returned no matching rows.':
            #     pass
            # else:
            #     try:
            #         m= driver.find_elements(By.CLASS_NAME,'title.variable-left-margin.with-description')
            #     except:
            #         m = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin')
            #     for j in m:
            #         js_code = "arguments[0].scrollIntoView();"
            #         driver.execute_script(js_code, m)
            #     title = m.text
            #     m.click()
            #     try:
            #         more_options = driver.find_elements(By.NAME,'expand')[1]
            #         more_options.click()
            #     except:
            #         more_options = driver.find_elements(By.NAME,'expand')[0]
            #         more_options.click()
            #     time.sleep(1)
            #     download = driver.find_element(By.XPATH,"//*[contains(text(), 'Download Data')]")#more_options.find_elements(By.CLASS_NAME,'text')[1]
            #     download.click()
            #     try:
            #         source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data source')]")[0].text
            #     except:
            #         source = driver.find_elements(By.XPATH,"//*[contains(text(), 'Data Source')]")[0].text
            # time.sleep(1)

[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS\\AfghanistanDonorDependencyPreventionInterventions_2023-5-15_1607.csv' -> 'C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS\\Afghanistan Afghanistan - Donor Dependency - Prevention Interventions.csv'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS\\AfghanistanDonorDependencyTreatmentInterventions_2023-5-16_0843.csv' -> 'C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS\\Afghanistan Afghanistan - Donor Dependency - Treatment Interventions.csv'
[WinError 3] The system cannot find the path specified: 'C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS\\AfghanistanInternationaldisbursementsforHIVAIDS_2023-5-15_1614.csv' -> 'C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS\\Afghanistan Afghanistan - International disbursements for HIV/AIDS.csv'
[WinError 87] The paramet

KeyboardInterrupt: 

In [151]:
file = max([base_path + "\\" + f for f in os.listdir(base_path)],key=os.path.getctime)
os.rename(file,f'{base_path}\\{i} {title}.csv')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS\\AfghanistanReportedexpendituresonHIVfromallfundingsources_2023-5-15_1025.csv' -> 'C:\\Users\\10235555\\Documents\\Dataportal Data\\UNAIDS\\<selenium.webdriver.remote.webelement.WebElement (session="ab22820fe3179515f035fda2a80c97cc", element="21D3F056D116B1D316BE304BFC279C91_element_735")> Afghanistan - Reported expenditures on HIV from all funding sources.csv'

In [136]:
iframe = driver.find_element(By.ID,'frameIt')
driver.switch_to.frame(iframe)
# m = box.find_element(By.CLASS_NAME, 'title.variable-left-margin.with-description')

In [110]:
try:
    m_2 = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin.with-description')[3]
except:
    m_2 = driver.find_elements(By.CLASS_NAME,'title.variable-left-margin')[3]
js_code = "arguments[0].scrollIntoView();"
driver.execute_script(js_code, m_2)
title = m_2.text
m_2.click()
more_options_2 = driver.find_elements(By.NAME,'expand')[3]
more_options_2.click()

In [162]:
driver.find_elements(By.NAME,'expand')[0].click()

In [156]:
m= driver.find_element(By.CLASS_NAME,'title.variable-left-margin')

In [143]:
m= driver.find_element(By.CLASS_NAME,'title.variable-left-margin.with-description')
js_code = "arguments[0].scrollIntoView();"
driver.execute_script(js_code, m)
time.sleep(2)
m.click()

In [84]:
content = driver.find_element(By.CLASS_NAME,'ui.five.column.relaxed.equal.height.divided.stackable.grid')
list_links = content.find_element(By.CLASS_NAME,'ui.link.list')
topic = content.find_element(By.TAG_NAME,'h4')
item = list_links.find_element(By.CLASS_NAME,'item')
item.click()
time.sleep(5)

In [85]:
driver.get('https://hivfinancial.unaids.org/hivfinancialdashboards.html')
time.sleep(2)
country = driver.find_element(By.ID,'countryAHref')
country.click()
time.sleep(2)

In [22]:
more_options = driver.find_element(By.CLASS_NAME,'alert-flex-wrapper.chart')
driver.execute_script("arguments[0].style.display = 'block';", more_options)

In [18]:
'widget-container dashboard-item-container widget-12107822 left-gutter redesign-chart-container chart-container-resizer ui-resizable ui-draggable'.replace(' ','.')

'widget-container.dashboard-item-container.widget-12107822.left-gutter.redesign-chart-container.chart-container-resizer.ui-resizable.ui-draggable'

In [7]:
iframe = driver.find_element(By.ID,'frameIt')
driver.switch_to.frame(iframe)
# driver.switch_to.default_content()

In [75]:
m= driver.find_element(By.CLASS_NAME,'title.variable-left-margin.with-description')
m.click()
more_options = driver.find_elements(By.NAME,'expand')[1]
more_options.click()
download = more_options.find_elements(By.CLASS_NAME,'text')[1]
download.click()